# Function Calling 简单示例

In [ ]:
! pip install openai

In [1]:
import json
from openai import OpenAI

client = OpenAI()

# 定义函数列表，同时在代码里实现该函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "analyze_loki_log",
            "description": "从 Loki 获取日志",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_str": {
                        "type": "string",
                        "description": 'Loki 查询字符串，例如：{app="grafana"} |= "Error"',
                    },
                },
                "required": ["query_str"],
            },
        },
    }
]

def analyze_loki_log(query_str):
    print("调用 analyze_loki_log 函数，入参：", query_str)
    print("进入函数调用获取日志")
    return "没有找到相关日志"

user_input = "查看 app=grafana 且关键字包含 Error 的日志"
# 系统提示语和用户消息
message = [
    {
        "role": "system",
        "content": "你是一个 Loki 日志分析助手，你可以帮助用户分析 Loki 日志，你可以调用多个函数来帮助用户完成任务，并最终尝试分析错误产生的原因",
    },
    {
        "role": "user",
        "content": user_input,
    },
]

# 步骤1：你的代码调用大模型 API，并将系统提示语、用户输入、函数列表及其功能描述一并传递给大模型
response = client.chat.completions.create(
    model="gpt-4o",
    messages=message,
    tools=tools,
    tool_choice="auto",
)
# 步骤2：大模型根据需求决定调用几个或多个方法来完成任务
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
response_message = response.choices[0].message

# 步骤 3：大模型返回给业务要调用的函数以及函数的入参消息
print("\nChatGPT want to call function: ", response_message)


ChatGPT want to call function:  ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ovg09I5nKPmTw6Sb7qbaX9es', function=Function(arguments='{"query_str":"{app=\\"grafana\\"} |= \\"Error\\""}', name='analyze_loki_log'), type='function')])


In [2]:
# 解析函数调用消息，如果有多个函数调用，则循环调用
tool_calls = response_message.tool_calls
for tool_call in tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    # 步骤4：你的业务代码用大模型给的参数和入参自行调用函数
    function_response = globals()[function_name](**function_args)
    print("函数返回结果: ",function_response)

    # 步骤5：用调用结果再次调用大模型 API，让大模型生成语义化的回复

    """
    注意再次调用大模型时，需传递所有的历史消息，包括系统消息、用户消息、函数调用消息、函数调用返回
    """
    # 加入刚才的系统提示语
    new_message = [
        {
            "role": "system",
            "content": "基于函数调用结果回答问题，如果函数调用结果不足以回答问题，不要编造，就说不清楚、不知道。",
        }
    ]
    # 把用户消息也加入进来
    new_message.append(
        {
            "role": "user",
            "content": user_input,
        }
    )
    # 把前面 LLM 返回的函数调用消息也加进来
    new_message.append(
        {
            "role": "assistant",
            "tool_calls": tool_calls,
        }
    )
    # 最后是函数回复消息
    new_message.append(
        {
            "tool_call_id": tool_call_id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )
    # 再次调用大模型，得到最终回复
    final_response = client.chat.completions.create(
        model="gpt-4o",
        messages=new_message,
    )
    print("第五步-最终推理: ",final_response.choices[0].message.content)
    


调用 analyze_loki_log 函数，入参： {app="grafana"} |= "Error"
进入函数调用获取日志
函数返回结果:  没有找到相关日志
第五步-最终推理:  没有找到包含“Error”关键字的 Grafana 日志。
